Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# Set Cover problem

See: https://en.wikipedia.org/wiki/Set_cover_problem

In [83]:
import random
from itertools import accumulate
import numpy as np
from matplotlib import pyplot as plt

from icecream import ic

## Reproducible Initialization

If you want to get reproducible results, use `rng` (and restart the kernel); for non-reproducible ones, use `np.random`.

In [84]:
# total number of elements in the universe set
UNIVERSE_SIZE = 100_000
# total number of subsets
NUM_SETS = 10_000
# each element of the universe set is covered by 30% of the subsets
DENSITY = 0.3

# number generator
rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)]))

In [85]:
# DON'T EDIT THESE LINES!

SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
# this iteration checks wether there is a column of SETS that has no value at True (i.e. it check that there is 
# any element that is not covered by any subset)
for s in range(UNIVERSE_SIZE):
    if not np.any(SETS[:, s]):
        # if there is one column of SETS all set to false, we set to true a random element (i.e. a random row)
        SETS[np.random.randint(NUM_SETS), s] = True

# array containing the cost associated to each subset
COSTS = np.power(SETS.sum(axis=1), 1.1)

## Helper Functions

In [86]:
def valid(solution):
    """ Checks wether solution is valid (ie. covers all universe) """
    return np.all(np.logical_or.reduce(SETS[solution]))


def cost(solution):
    """ Returns the cost of a solution (to be minimized) """
    return COSTS[solution].sum()

## Have Fun!

In [ ]:
# A dumb solution of "all" sets
solution = np.full(NUM_SETS, True)
valid(solution), cost(solution)

In [ ]:
# A random solution with random 50% of the sets
solution = rng.random(NUM_SETS) < .5
valid(solution), cost(solution)

## Trivial solution: **Single Random Mutation Hill Climber**
This approach starts with a candidate solution which is the empty solution, i.e. no set is considered (every element of the array is set to `False`). The tweak function randomly flips one element of the current solution. The flipping is a one-way flipping (only from `False` to `True`). The iterative process keeps tweaking the current solution until it reaches a state of no improvement at which we empirically consider that no more optimization is possible (in the surrounding space) and, because of this, we obtained our final solution.

In [89]:
# definition of the candidate solution
candidate_solution = np.array([False] * NUM_SETS)

In [90]:
# this function computes the number of the elements of the universe set that are currently covered by one or more
# subsets of the current solution
def covered_elements(solution):
    selected_sets = SETS[solution, :]
    combined_elements = np.any(selected_sets, axis=0)
    num_covered_elements = np.sum(combined_elements)

    return num_covered_elements


# definition of the fitness function: 
# - maximization of the covered elements (in order not to keep the solution to the starting state)
# - minimization of the total cost of the solution
def fitness (solution):
    return (covered_elements(solution), -cost(solution))

In [91]:
# the tweak function takes a random element of the solution with value False and sets it to True
def tweak (current_solution):
    new_solution = current_solution.copy()
    if np.sum(current_solution) == NUM_SETS:
        return current_solution
    while True:
        rand_index = random.randint(0, (NUM_SETS - 1))
        if current_solution[rand_index] == False:
            break
    
    new_solution[rand_index] = True
    return new_solution

### Iterative process

In [ ]:
steps = 0
current_solution = candidate_solution

history = [float(fitness(candidate_solution)[0])]
best = current_solution
while True:
    steps += 1

    # the tweak is executed multiple times times looking for the best improvement
    for _ in range(10):
        solution = tweak(current_solution)
        if fitness(solution) > fitness(best):
            best = solution
    
    # if the previous tweeks resulted in no improvement, we assume to be in a state in which no immediate optimization
    # is possible (so we reached a local optimum)
    if np.array_equal(best, current_solution) and valid(best):
        break
    else:
        # if improvements of the solution are possible we take the best one
        current_solution = best
        history.append(float(fitness(current_solution)[0]))
    

ic(steps)
ic(cost(current_solution))

### Iteration plot

In [ ]:
plt.figure(figsize=(14, 8))
plt.plot(
    range(len(history)),
    list(accumulate(history, max)),
    color="red",
)
_ = plt.scatter(range(len(history)), history, marker=".")

## Second approach: **Multiple Random Mutation Hill Climber**
This solution uses a simulated annealing approach implementing a dinamic temperature value, which allows to balance exploration (big jumps) if the solution worsens or exploutation (small jumps) if the solution is improving.

In [94]:
# definition of the candidate solution
candidate_solution = np.random.rand(NUM_SETS) < 0.3

In [95]:
# definition of the fitness function (same as before)
def fitness (solution):
    return (covered_elements(solution), -cost(solution))

In [96]:
# definition of the tweak function
temperature = 0.5

def tweak (current_solution):
    mask = rng.random(NUM_SETS) < temperature
    
    if not np.any(mask):
        mask[np.random.randint(NUM_SETS)] = True

    new_solution = np.logical_xor(current_solution, mask)
    return new_solution

### Iterative process

In [ ]:
steps = 0
current_solution = candidate_solution
history = [float(fitness(candidate_solution)[1])]

BUFFER_SIZE = 10
buffer = list()

for _ in range(1000):
    steps += 1
    
    solution = tweak(current_solution)
    history.append(float(fitness(solution)[1]))

    buffer.append(fitness(solution) > fitness(current_solution))
    buffer = buffer[-BUFFER_SIZE:]

    if sum(buffer) > BUFFER_SIZE / 2:
        temperature *= 1.5
    elif sum(buffer) < BUFFER_SIZE / 2:
        temperature /= 1.5


    if fitness(solution) > fitness(current_solution):
        current_solution = solution
    

ic(steps)
ic(cost(current_solution))

### Iteration plot

In [ ]:
plt.figure(figsize=(14, 8))
plt.plot(
    range(len(history)),
    list(accumulate(history, max)),
    color="red",
)
_ = plt.scatter(range(len(history)), history, marker=".")